In [ ]:
import pandas as pd
import numpy as np
import bs4
from openpyxl import load_workbook
from urllib.request import urlopen
import csv

pd.set_option('display.max_rows',500)  #데이터프레임 truncation 방지
class stock_evaluation:

    def get_html(self,jcode):     #재무재표 크롤링
        self.jcode = jcode
        url = 'http://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A%s' % jcode
        html = urlopen(url).read()
        html = bs4.BeautifulSoup(html,'lxml')
        table = html.find('div',{'id':'highlight_D_Y'})
        crt_data = html.find('div',{'id':'corp_group2'}).find_all('dl')
        self.crt_PER = crt_data[0].find_all('dd')[1].text   #PER
        self.crt_around_PER = crt_data[4].find_all('dd')[1].text   #업종 PER
        self.crt_PBR = float(crt_data[6].find_all('dd')[1].text)
        self.crt_price = int(html.find('span',{'id' : 'svdMainChartTxt11'}).text.replace(',',''))
        self.name = html.find('h1',{'id':'giName'}).text
        
        date_col = table.find_all('tr')[1].find_all('th')[:5]
        date_col=list(i.text for i in date_col) #dataframe의 column

        j = table.find_all('td',{'class','r'})
        tr = table.find_all('tr')[18:]
        self.df = pd.DataFrame()
        
    
        for i in tr:        #테이블 항목 가져오기
            category = i.find('span',{'class':'txt_acd'})
            if(category==None):
                category=i.find('th')
            category = category.text.strip()
            value_list = []
            j = i.find_all('td',{'class','r'})[:5]
        
            for value in j:
                temp = value.text.replace(',','').strip()
                try:
                    temp = float(temp)
                    value_list.append(temp)
                except:
                    value_list.append(0)
                data = {category:value_list}
                data = pd.DataFrame(data)
            self.df = pd.concat([self.df,data],axis=1)

        self.df.index=date_col
        return self.df
    
    def get_excel_high_dividend(self):    #고배당주 엑셀 read
        load_wb = load_workbook("stock_div.xlsx", data_only=True)
        load_ws = load_wb['Sheet1']
        row_num = 0
        jcode=[]
        for row in load_ws.rows:
            row_num+=1
            if row_num==1:
                continue
            elif row_num==100:   #100개만
                break
            jcode.append(row[2].value)
        return jcode

    def get_excel_siga_top(self):  #시가총액 엑셀 read
        load_wb = open('siga.csv','r',encoding='utf-8')
        load_ws = csv.reader(load_wb)
        row_num = 0
        jcode=[]
        for row in load_ws:
            row_num+=1
            if row_num==1:
                continue
            jcode.append(row[1])
        return jcode
        
        
        
    def df_extension(self):       #데이터테이블 확장(2025년까지)
        for i in range(6):
            self.ROE_mean = self.df['ROE'].mean()
            new_eps = self.df['BPS'][-1]*self.ROE_mean/100
            new_bps = new_eps + self.df['BPS'][-1]
            PER = self.df['PER'][-1]
            PBR = self.df['PBR'][-1]
            stock_ret = self.df['배당수익률'][-1]
            self.df.loc['202'+str(i)+'/12'] = [self.df['ROA'].mean(),self.ROE_mean,new_eps,new_bps, np.nan, PER, PBR, np.nan, stock_ret]
        return self.df
    
    def report(self):
        self.new_price = int(self.df['BPS'][-1]/((1.15)**5))

        self.all_datas = pd.DataFrame(index = ['종목이름','종목코드','평균ROE','PER','업종PER','현재가치','주가','평가'],
                         data = [self.name,self.jcode,self.ROE_mean,self.crt_PER,self.crt_around_PER,self.new_price,self.crt_price,\
                                 '적합' if self.new_price>=1.5*self.crt_price else '부적합']).T
        return self.all_datas
        
    
if __name__=="__main__":
    print("주식종목 가치 평가 프로그램입니다.\n")
    se_func = stock_evaluation()
    ch1 = input("1. 개별주식종목 조회, 2.고배당주 top100 3.시가총액 상위 100개 기업 \n")
    report_df = pd.DataFrame()
    
    if(ch1=='1'):
        while(True):
            jcode = input("종목코드를 입력하세요 : ")
            se_func.get_html(jcode)   
            se_func.df_extension() 
            se_func.report()
            report_df = pd.concat([report_df,se_func.all_datas],axis=0)
            report_df.reset_index(drop=True,inplace=True)
            display(report_df)
            ch2 = input("계속 조회하려면 Y, else N")
            if(ch2=='n' or ch2=='N'):
                break
            elif(ch2=='Y' or ch2=='y'):
                continue
            else:
                print("key error")
                break
                
    elif(ch1=='2'):
        for i in range(200): #100개만 조회
            try:
                se_func.get_html(se_func.get_excel_high_dividend()[i])   
                se_func.df_extension() 
                se_func.report()
                report_df = pd.concat([report_df,se_func.all_datas],axis=0)
                report_df.reset_index(drop=True,inplace=True)
            except:
                pass
        display(report_df)
    elif(ch1=='3'):
        for i in range(1000) : #100개만 조회
            try:
                se_func.get_html(se_func.get_excel_siga_top()[i])
                se_func.df_extension()
                se_func.report()
                report_df = pd.concat([report_df, se_func.all_datas], axis=0)
                report_df.reset_index(drop=True, inplace=True)
            except:
                pass
        display(report_df)
    




주식종목 가치 평가 프로그램입니다.

1. 개별주식종목 조회, 2.고배당주 top100 3.시가총액 상위 100개 기업 
1
종목코드를 입력하세요 : 094850


,종목이름,종목코드,평균ROE,PER,업종PER,현재가치,주가,평가
0,참좋은여행,094850,15.622,3.39,-,8755,5550,적합


계속 조회하려면 Y, else Ny
종목코드를 입력하세요 : 002350


,종목이름,종목코드,평균ROE,PER,업종PER,현재가치,주가,평가
0,참좋은여행,094850,15.622,3.39,-,8755,5550,적합
1,넥센타이어,002350,10.758,4.92,47.92,13720,5580,적합


In [4]:
for i in range(len(report_df)):
    if report_df.loc[i]['평가']=='적합' and report_df.loc[i]['PER']<report_df.loc[i]['업종PER']:
        display(pd.DataFrame(report_df.iloc[i]).T)

       # [0],report_df.loc[i][1], report_df.loc[i][2], report_df.loc[i][3], report_df.loc[i][4],report_df.loc[i][5],\
        #     report_df.loc[i][6]

,종목이름,종목코드,평균ROE,PER,업종PER,현재가치,주가,평가
16,SK,034730,21.648,20.82,7.05,420197,210500,적합


,종목이름,종목코드,평균ROE,PER,업종PER,현재가치,주가,평가
18,KB금융,105560,8.226,4.69,7.05,76220,37150,적합


,종목이름,종목코드,평균ROE,PER,업종PER,현재가치,주가,평가
19,신한지주,055550,8.856,4.06,7.05,66924,28450,적합


,종목이름,종목코드,평균ROE,PER,업종PER,현재가치,주가,평가
30,하나금융지주,086790,7.32,3.54,7.05,71914,28200,적합


,종목이름,종목코드,평균ROE,PER,업종PER,현재가치,주가,평가
37,우리금융지주,316140,0,3.11,7.05,14806,8390,적합


...

,종목이름,종목코드,평균ROE,PER,업종PER,현재가치,주가,평가
913,DB금융투자,016610,3.05,3.30,6.54,9920,3770,적합


,종목이름,종목코드,평균ROE,PER,업종PER,현재가치,주가,평가
914,삼영전자,005680,2.61,19.62,22.91,14054,8390,적합


,종목이름,종목코드,평균ROE,PER,업종PER,현재가치,주가,평가
917,KTB투자증권,030210,7.948,3.75,6.54,6370,2660,적합


,종목이름,종목코드,평균ROE,PER,업종PER,현재가치,주가,평가
918,SIMPAC,009160,8.43,16.69,53.41,5642,2420,적합


,종목이름,종목코드,평균ROE,PER,업종PER,현재가치,주가,평가
946,AJ네트웍스,095570,6.938,3.22,53.87,5057,3160,적합


In [7]:
abc = stock_evaluation()
abc.get_html('060310')
abc.df_extension()
abc.report()

,종목이름,종목코드,평균ROE,PER,업종PER,현재가치,주가,평가
0,3S,060310,-8.126,-,230.40,222,2265,부적합


In [50]:
se_func.get_html(se_func.get_excel_siga_top()[99])
se_func.df_extension()
se_func.report()
report_df = pd.concat([report_df, se_func.all_datas], axis=0)
report_df.reset_index(drop=True, inplace=True)
report_df

,종목이름,종목코드,평균ROE,PER,업종PER,현재가치,주가,평가
0,펄어비스,263750,92.666,15.24,45.03,1194509,184700,적합


In [14]:
url = 'http://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A123890'
html = urlopen(url).read()
html = bs4.BeautifulSoup(html,'lxml')
table = html.find('div',{'id':'highlight_D_Y'})
crt_data = html.find('div',{'id':'corp_group2'}).find_all('dl')
PBR = crt_data[6].find_all('dd')
PBR[1].text

'0.56'